In [1]:
import cv2
import keras
import keras.backend as K
from keras.losses import binary_crossentropy
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.generic_utils import get_custom_objects
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm

Using TensorFlow backend.
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/conda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

## Utility Functions

### Loss performance metrics and loss function

In [2]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
def dice_loss(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = y_true_f * y_pred_f
    score = (2. * K.sum(intersection) + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return 1. - score
def bce_dice_loss(y_true, y_pred):
    return binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)

### RLE Mask encoding and decoding

In [3]:
def mask2rle(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels= img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def rle2mask(rle, input_shape):
    width, height = input_shape[:2]
    
    mask= np.zeros( width*height ).astype(np.uint8)
    
    array = np.asarray([int(x) for x in rle.split()])
    starts = array[0::2]
    lengths = array[1::2]

    current_position = 0
    for index, start in enumerate(starts):
        mask[int(start):int(start+lengths[index])] = 1
        current_position += lengths[index]
        
    return mask.reshape(height, width).T

def build_masks(rles, input_shape):
    depth = len(rles)
    masks = np.zeros((*input_shape, depth))
    
    for i, rle in enumerate(rles):
        if type(rle) is str:
            masks[:, :, i] = rle2mask(rle, input_shape)
    
    return masks

def build_rles(masks):
    width, height, depth = masks.shape
    
    rles = [mask2rle(masks[:, :, i])
            for i in range(depth)]
    
    return rles

### Data Generator

In [4]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, df, target_df=None, mode='fit',
                 base_path='../input/severstal-steel-defect-detection/train_images',
                 batch_size=32, dim=(256, 1600), n_channels=1,
                 n_classes=4, random_state=2019, shuffle=True):
        self.dim = dim
        self.batch_size = batch_size
        self.df = df
        self.mode = mode
        self.base_path = base_path
        self.target_df = target_df
        self.list_IDs = list_IDs
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.random_state = random_state
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_batch = [self.list_IDs[k] for k in indexes]
        
        X = self.__generate_X(list_IDs_batch)
        
        if self.mode == 'fit':
            y = self.__generate_y(list_IDs_batch)
            return X, y
        
        elif self.mode == 'predict':
            return X

        else:
            raise AttributeError('The mode parameter should be set to "fit" or "predict".')
        
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.seed(self.random_state)
            np.random.shuffle(self.indexes)
    
    def __generate_X(self, list_IDs_batch):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim, self.n_channels))
        
        # Generate data
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            img_path = f"{self.base_path}/{im_name}"
            img = self.__load_grayscale(img_path)
            
            # Store samples
            X[i,] = img

        return X
    
    def __generate_y(self, list_IDs_batch):
        y = np.empty((self.batch_size, *self.dim, self.n_classes), dtype=int)
        
        for i, ID in enumerate(list_IDs_batch):
            im_name = self.df['ImageId'].iloc[ID]
            image_df = self.target_df[self.target_df['ImageId'] == im_name]
            
            rles = image_df['EncodedPixels'].values
            masks = build_masks(rles, input_shape=self.dim)
            
            y[i, ] = masks

        return y
    
    def __load_grayscale(self, img_path):
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
        img = img.astype(np.float32) / 255.
        img = np.expand_dims(img, axis=-1)

        return img

## Load Dataframe and Models

In [5]:
sub_df = pd.read_csv('../input/editeddata/sub_df.csv') #preprocessed sample submission
test_imgs = pd.read_csv('../input/editeddata/test_imgs.csv') #list of unique ImageId in test dataset

In [6]:
mobilenet = load_model('../input/severstal-model/cls-model-mobilenet.h5')
mobilenet.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
mobilenetv2_1.00_256 (Model) (None, 8, 8, 1280)        2257984   
_________________________________________________________________
global_average_pooling2d_1 ( (None, 1280)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 1281      
Total params: 2,259,265
Trainable params: 2,225,153
Non-trainable params: 34,112
_________________________________________________________________


In [8]:
custom_objects = custom_objects={
    'dice_coef': dice_coef,
    'dice_loss': dice_loss,
    'bce_dice_loss': bce_dice_loss}
unet_model_path = '../input/severstal-model/35e-model.h5'
unet = load_model(unet_model_path, custom_objects=custom_objects)
unet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 256, 1600, 1) 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 1600, 8) 80          input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 256, 1600, 8) 584         conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 128, 800, 8)  0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
conv2d_3 (

## Create test generator

In [9]:
def create_test_gen(batch_size=64):
    return ImageDataGenerator(rescale=1/255.).flow_from_dataframe(
        test_imgs,
        directory='../input/severstal-steel-defect-detection/test_images',
        x_col='ImageId',
        class_mode=None,
        target_size=(256, 256),
        batch_size=batch_size,
        shuffle=False
    )

## Perform Removal

In [10]:
test_gen = create_test_gen()

test_pred = mobilenet.predict_generator(
    test_gen,
    steps=len(test_gen),
    verbose=1)
test_imgs['allMissing'] = test_pred

filtered_test_imgs = test_imgs[test_imgs['allMissing'] < 0.5]
print(filtered_test_imgs.shape)
filtered_test_imgs.head()

Found 5506 validated image filenames.
87/87 [==============================] - 74s 849ms/step
(1496, 2)


,ImageId,allMissing
3,003c5da97.jpg,0.057570
4,0042e163f.jpg,0.046502
6,00513039a.jpg,0.002631
14,00bbcd9af.jpg,0.040701
19,0109b68ec.jpg,0.201878


`filtered_sub_df` contains all of the images with at least one mask. `null_sub_df` contains all the images with exactly 4 missing masks.

In [11]:
filtered_mask = sub_df['ImageId'].isin(filtered_test_imgs["ImageId"].values)
defect_df = sub_df[filtered_mask].copy()
clean_df = sub_df[~filtered_mask].copy()
clean_df['EncodedPixels'] = clean_df['EncodedPixels'].apply(lambda x: ' ')

defect_df.reset_index(drop=True, inplace=True)
filtered_test_imgs.reset_index(drop=True, inplace=True)

print(defect_df.shape)
print(clean_df.shape)

defect_df.head()

(5984, 3)
(16040, 3)


,ImageId_ClassId,EncodedPixels,ImageId
0,003c5da97.jpg_1,NaN,003c5da97.jpg
1,003c5da97.jpg_2,NaN,003c5da97.jpg
2,003c5da97.jpg_3,NaN,003c5da97.jpg
3,003c5da97.jpg_4,NaN,003c5da97.jpg
4,0042e163f.jpg_1,NaN,0042e163f.jpg


# Step 2: Predict masks using U-Net++

## Inference on test set

In [12]:
segpred_df = []

for i in range(0, filtered_test_imgs.shape[0], 300):
    batch_idx = list(
        range(i, min(filtered_test_imgs.shape[0], i + 300))
    )
    
    test_generator = DataGenerator(
        batch_idx,
        df=filtered_test_imgs,
        shuffle=False,
        mode='predict',
        base_path='../input/severstal-steel-defect-detection/test_images',
        target_df=defect_df,
        batch_size=1,
        n_classes=4
    )
    
    batch_pred_masks = unet.predict_generator(
        test_generator, 
        workers=1,
        verbose=1,
        use_multiprocessing=False
    )
    
    for j, b in tqdm(enumerate(batch_idx)):
        filename = filtered_test_imgs['ImageId'].iloc[b]
        image_df = defect_df[defect_df['ImageId'] == filename].copy()
        
        pred_masks = batch_pred_masks[j, ].round().astype(int)
        pred_rles = build_rles(pred_masks)
        
        image_df['EncodedPixels'] = pred_rles
        segpred_df.append(image_df)

300/300 [==============================] - 7s 23ms/step


300it [00:04, 60.74it/s]

  5/300 [..............................] - ETA: 4s

300/300 [==============================] - 4s 14ms/step


300it [00:04, 62.10it/s]

...

300it [00:04, 62.57it/s]


300/300 [==============================] - 5s 16ms/step


300it [00:04, 61.40it/s]


296/296 [==============================] - 4s 14ms/step


296it [00:04, 63.33it/s]


In [13]:
segpred_df = pd.concat(segpred_df)
final_submission_df = pd.concat([segpred_df, clean_df])

print(segpred_df.shape)
print(final_submission_df.shape)

final_submission_df.head()

(5984, 3)
(22024, 3)


,ImageId_ClassId,EncodedPixels,ImageId
0,003c5da97.jpg_1,,003c5da97.jpg
1,003c5da97.jpg_2,,003c5da97.jpg
2,003c5da97.jpg_3,198812 1 199324 3 199580 2 199836 3 200093 3 2...,003c5da97.jpg
3,003c5da97.jpg_4,,003c5da97.jpg
4,0042e163f.jpg_1,,0042e163f.jpg


In [14]:
final_submission_df[['ImageId_ClassId', 'EncodedPixels']].to_csv('mobile-35eunet-submission.csv', index=False)